# API REST et requete sur endpoint: vieille methode peu flexible
* L'api est limité , impossible de recuperer plus de 3 reviews par business

In [1]:
#client id: PtmNC-6eafkEZj0qwH7C5A
# API key: ROAZmQQQUYoPK2vBEjuxWqJu4SEGGpVZacWIolGmFYgMAZsg-BIpO0g71j3nf3QcLF4yW5g9jVkJOoDOSmi5FAfK86Eb_9qnMGTrQngsRTYWV4dA2H8xtMsF2UDpYXYx
# App Name: restau
# Daily API limit

# GraphQL (beta) ss the regular API
# To authenticate API calls with the API Key, set the Authorization HTTP header value as Bearer API_KEY

In [2]:
#elegant and simple HTTP library for Python
import requests
import pandas as pd

In [110]:
url='https://api.yelp.com/v3/businesses/search'
key='ROAZmQQQUYoPK2vBEjuxWqJu4SEGGpVZacWIolGmFYgMAZsg-BIpO0g71j3nf3QcLF4yW5g9jVkJOoDOSmi5FAfK86Eb_9qnMGTrQngsRTYWV4dA2H8xtMsF2UDpYXYx'
headers={'Authorization':'Bearer %s' % key}

parameters ={'location': "New York City",
            'categories': 'restaurants',
            'sort_by':'review_count', 
            'limit':3}

In [111]:
requests.get(url, headers=headers, params=parameters).json()

{'businesses': [{'id': 'V7lXZKBDzScDeGB8JmnzSA',
   'alias': 'katzs-delicatessen-new-york',
   'name': "Katz's Delicatessen",
   'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/7Yn37rOW4VQDI396jPPoyA/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/katzs-delicatessen-new-york?adjust_creative=PtmNC-6eafkEZj0qwH7C5A&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=PtmNC-6eafkEZj0qwH7C5A',
   'review_count': 13254,
   'categories': [{'alias': 'delis', 'title': 'Delis'},
    {'alias': 'sandwiches', 'title': 'Sandwiches'},
    {'alias': 'soup', 'title': 'Soup'}],
   'rating': 4.0,
   'coordinates': {'latitude': 40.722237, 'longitude': -73.9875259},
   'transactions': ['delivery', 'pickup'],
   'price': '$$',
   'location': {'address1': '205 E Houston St',
    'address2': '',
    'address3': '',
    'city': 'New York',
    'zip_code': '10002',
    'country': 'US',
    'state': 'NY',
    'display_address': ['205 E Houston St', 'New York, NY 10002']

In [73]:
def getbiz(parameters):
    url='https://api.yelp.com/v3/businesses/search'
    headers={'Authorization':'Bearer %s' % key}
    
    response = requests.get(url, headers=headers, params=parameters)
    data=response.json()['businesses']
    
    ids=[]
    names=[]
    ratings=[]
    pricing=[]
    cnt=[]
    address=[]
    city=[]
    country=[]
    for i in range(len(data)):
        ids.append(data[i]['id'])
        names.append(data[i]['name'])
        ratings.append(data[i]['rating'])
        city.append(data[i]['location']['city'])
        country.append(data[i]['location']['country'])
        address.append(data[i]['location']['address1'])
        try:
            pricing.append(len(data[i]['price']))
        except:
            pricing.append(none)
        cnt.append(data[i]['review_count'])
    biz={'id':ids, 'name':names,'rating':ratings,'pricing':pricing,'num_review':cnt,'country':country,'city':city,'street_address':address}
    return pd.DataFrame(biz)


#limité à 3 reviews par biz
def getreviews(biz):
    headers={'Authorization':'Bearer %s' % key}
    
    id_biz=[]
    id_review=[]
    ratings=[]
    user_ids=[]
    texts=[]
    creation=[]
        
    for c,i in enumerate(biz.id):        
        url='https://api.yelp.com/v3/businesses/'+i+'/reviews'
        response = requests.get(url, headers=headers)
        data=response.json()['reviews']
        
        for j in range(len(data)):
            id_biz.append(i)
            id_review.append(data[j]['id'])
            ratings.append(data[j]['rating'])
            user_ids.append(data[j]['user']['id'])
            texts.append(data[j]['text'])
            creation.append(data[j]['time_created'])
    rev={'id_biz':id_biz,'id_review':id_review, 'rating':ratings,'user_id':user_ids,'text':texts,'created_time':creation}
    return pd.DataFrame(rev)

In [74]:
parameters ={'location': "New York City",
            'categories': 'restaurants',
            'sort_by':'review_count', 
            'limit':3}
biz=getbiz(parameters)
rev=getreviews(biz)

In [75]:
biz

,id,name,rating,pricing,num_review,country,city,street_address
0,V7lXZKBDzScDeGB8JmnzSA,Katz's Delicatessen,4.0,2,13254,US,New York,205 E Houston St
1,44SY464xDHbvOcjDzRbKkQ,Ippudo NY,4.0,2,10163,US,New York,65 4th Ave
2,xEnNFXtMLDF5kZDxfaCJgA,The Halal Guys,4.0,1,9740,US,New York,W 53rd St


In [76]:
rev

,id_biz,id_review,rating,user_id,text,created_time
0,V7lXZKBDzScDeGB8JmnzSA,YErmxd8Wq5js3APNScpSRw,4,2yhyR8WMLGaShrrZ2BQfcQ,1. Pastrami sandwich was amazing\n2. Little cr...,2021-12-18 22:09:06
1,V7lXZKBDzScDeGB8JmnzSA,KOVMt-uFUDNf6p52eL7AcQ,5,TU-aeCjEdO-RZ1dZt1mxjQ,Muy buena experiencia pónganlo en su lista de ...,2021-12-30 13:44:58
2,V7lXZKBDzScDeGB8JmnzSA,hFppQiUJ6M-JCwrcwAF95g,5,GTbrhR4neUkotAVbP1Xy0Q,"Great place love their pastrami on rye bread, ...",2021-12-26 06:00:36
3,44SY464xDHbvOcjDzRbKkQ,IClV7xq3Lc0cCuW8zfhwxw,5,IcjPM9m9MHCDpkuwBXqvQA,Our friends had recommended this restaurant fo...,2022-01-18 19:11:01
4,44SY464xDHbvOcjDzRbKkQ,fxls6rgPRXOWhDnyqbm1rA,3,xQLbzbrBh4DKkaPrQQIXVQ,"My friend and I came here for lunch, knowing t...",2022-01-17 18:51:08
5,44SY464xDHbvOcjDzRbKkQ,7m-LLrzag-XIbYyoicDmwA,4,UfiGEp8owldl-K2y_0lhtg,What I ordered | Akamaru Modern Ramen \n\nTast...,2021-12-31 13:29:11
6,xEnNFXtMLDF5kZDxfaCJgA,v3JPH5YCIJT5LiQsBGze7A,5,iSYvygYFr6ix4cIcnxP7HQ,It is very great great meat great rice and bes...,2021-12-17 16:03:01
7,xEnNFXtMLDF5kZDxfaCJgA,Ybd21AjqutbGoNlMDWYNjg,3,ccCp2gy20w2atwo-Vfz1iw,"Delicious,you should try...but no sitting plac...",2021-12-01 19:53:20
8,xEnNFXtMLDF5kZDxfaCJgA,1JL6Buiz_JDBAvchzqB_Jw,4,45yyegDSmWnS-GF9a4j2hw,"A little pricey for my liking, but over all th...",2021-12-07 08:08:28


# GRAPHQL (efficience et flexibilité via facebook)
* limité à 3 reviews par buis et à 50 requetes en une fois (mais on oue avec l'offset pour en avoir un peu plus jusqu a une certaine limite certainement)
* https://www.jesuisundev.com/comprendre-graphql-en-5-minutes/
* https://reqbin.com/req/python/c-xgafmluu/convert-curl-to-python-requests

In [179]:
import requests
from requests.structures import CaseInsensitiveDict

import json
import pandas as pd

#curl -X POST -H "Authorization: Bearer ACCESS_TOKEN" -H "Content-Type: application/graphql" https://api.yelp.com/v3/graphql
url = "https://api.yelp.com/v3/graphql"

headers = CaseInsensitiveDict()
headers["Authorization"] = "Bearer %s" % key
headers["Content-Type"] = "application/graphql"

data = """

{
    search(location: "New York City",
            categories: "restaurants",
            sort_by:"review_count", 
            limit:50) {
        total
        business {
            id
            name
            rating
            location {
                city
                country
                address1
            }
            price
            review_count
            reviews {
                id
                rating
                user{
                    id
                }
                text
                time_created
            }
        }
    }
}
"""


#resp = requests.post(url, headers=headers, data=data)
#resp.json()
#print(resp.status_code)



In [180]:
def getgqldata():
    
    #init des listes qui devront faire la meme longueur pour le dataframe
    biz_id=[]
    biz_name=[]
    biz_rating=[]
    city=[]
    country=[]
    address=[]
    price=[]
    review_count=[]
    review_id=[]
    review_rating=[]
    user_id=[]          
    review_txt=[]               
    review_creation=[]
    
    #url/header/requete du curl graphql
    url = "https://api.yelp.com/v3/graphql"

    headers = CaseInsensitiveDict()
    headers["Authorization"] = "Bearer %s" % key
    headers["Content-Type"] = "application/graphql"
    
    #boucle sur les offset pour recup plus de 50 requetes de buisness
    for offset in range(0, 200, 50):
        datas = """

        {
            search(location: "New York City",
                    categories: "restaurants",
                    sort_by:"review_count", 
                    limit:50,
                    offset:"""+str(offset)+""") {
                total
                business {
                    id
                    name
                    rating
                    location {
                        city
                        country
                        address1
                    }
                    price
                    review_count
                    reviews {
                        id
                        rating
                        user{
                            id
                        }
                        text
                        time_created
                    }
                }
            }
        }
        """
        
        #on post la requete curl et on recupere la reponse en objet json
        data=requests.post(url=url, headers=headers, data=datas).json()['data']['search']['business']
    
    
        #on construit le dataframe à partir de dict en construisant des listes à partir des données json
        for i in range(len(data)):      
            for j in range(len(data[i]['reviews'])):
                biz_id.append(data[i]['id'])
                biz_name.append(data[i]['name'])
                biz_rating.append(data[i]['rating'])
                city.append(data[i]['location']['city'])
                country.append(data[i]['location']['country'])
                address.append(data[i]['location']['address1'])
                try:
                    price.append(len(data[i]['price']))
                except:
                    price.append(none)
                review_count.append(data[i]['review_count'])
        
                review_id.append(data[i]['reviews'][j]['id'])
                review_rating.append(data[i]['reviews'][j]['rating'])
                user_id.append(data[i]['reviews'][j]['user']['id'])
                review_txt.append(data[i]['reviews'][j]['text'])
                review_creation.append(data[i]['reviews'][j]['time_created'])

    rev={'biz_id':biz_id,'biz_name':biz_name,'biz_rating':biz_rating,'city':city,'country':country,
         'address':address,'price':price,'biz_review_count':review_count,'review_id':review_id,'review_rating':review_rating,
        'user_id':user_id,'review_txt':review_txt,'review_creation':review_creation}
    return pd.DataFrame(rev)

In [181]:
rev2=getgqldata()
rev2

,biz_id,biz_name,biz_rating,city,country,address,price,biz_review_count,review_id,review_rating,user_id,review_txt,review_creation
0,V7lXZKBDzScDeGB8JmnzSA,Katz's Delicatessen,4.0,New York,US,205 E Houston St,2,13255,YErmxd8Wq5js3APNScpSRw,4,2yhyR8WMLGaShrrZ2BQfcQ,1. Pastrami sandwich was amazing\n2. Little cr...,2021-12-18 22:09:06
1,V7lXZKBDzScDeGB8JmnzSA,Katz's Delicatessen,4.0,New York,US,205 E Houston St,2,13255,KOVMt-uFUDNf6p52eL7AcQ,5,TU-aeCjEdO-RZ1dZt1mxjQ,Muy buena experiencia pónganlo en su lista de ...,2021-12-30 13:44:58
2,V7lXZKBDzScDeGB8JmnzSA,Katz's Delicatessen,4.0,New York,US,205 E Houston St,2,13255,hFppQiUJ6M-JCwrcwAF95g,5,GTbrhR4neUkotAVbP1Xy0Q,"Great place love their pastrami on rye bread, ...",2021-12-26 06:00:36
3,44SY464xDHbvOcjDzRbKkQ,Ippudo NY,4.0,New York,US,65 4th Ave,2,10165,IClV7xq3Lc0cCuW8zfhwxw,5,IcjPM9m9MHCDpkuwBXqvQA,Our friends had recommended this restaurant fo...,2022-01-18 19:11:01
4,44SY464xDHbvOcjDzRbKkQ,Ippudo NY,4.0,New York,US,65 4th Ave,2,10165,fxls6rgPRXOWhDnyqbm1rA,3,xQLbzbrBh4DKkaPrQQIXVQ,"My friend and I came here for lunch, knowing t...",2022-01-17 18:51:08
...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,UKy0nNjMkOBEU3dOCH831w,Cuba Restaurant and Rum Bar,4.0,New York,US,222 Thompson St,2,1609,etX-TVZ-4wTSDe1TKmEG1Q,5,nz7NEC0jFMUK6wkGnMl11w,You cannot go wrong here. One of the best Lati...,2021-08-22 23:54:25
596,UKy0nNjMkOBEU3dOCH831w,Cuba Restaurant and Rum Bar,4.0,New York,US,222 Thompson St,2,1609,q1HBHrOHEoirS-njceV0mQ,2,Dp6HMJq7Jjnmb8V6ogs2rg,"I've come to Cuba restaurant for many years, a...",2021-08-09 18:26:36
597,puIxzUYl8hhVfWJRaMsZcA,The Mercer Kitchen,3.5,New York,US,99 Prince St,3,1605,uSuOisngAnZdJXNhZFGMBw,3,oQ2p7o4tbZheY2KuY3HamA,The ambience was really nice and had a good vi...,2021-12-24 16:56:22
598,puIxzUYl8hhVfWJRaMsZcA,The Mercer Kitchen,3.5,New York,US,99 Prince St,3,1605,LPesGxk14gJ1MnqN8vUKdw,4,K3NQul4_IAaS3PuUMztr-A,This place has the elegance and service of a M...,2021-12-27 20:31:52


In [187]:
rev2['review_txt']=rev2['review_txt'].str.replace('\n',' ')
rev2.to_csv('C:/Users/wesh/OCR/P6/req_yelp.csv',sep=';',index=False)
